#Notebook for Colletive Intelligence 2015
##Contact: NicolÃ¡s Della Penna me@nikete.com, Dhaval Adjodah, dhaval@mit.edu

In [9]:
%%javascript
IPython.load_extensions('usability/codefolding/main');
// code folding capability as per: https://github.com/ipython-contrib/IPython-notebook-extensions/wiki/Codefolding

<IPython.core.display.Javascript object>

In [ ]:
%%script R --no-save
## R script to download data and prepare features
# dhaval@mit.edu
# this script will download all data from the SciCast datamart, compute the necessary features and output it as a CSV. 
# This takes about 10min on an i7 CPU with 8gb RAM
# note: you will need to request your API_KEY_FROM_SCICAST as per https://scicast.files.wordpress.com/2014/10/scicast_datamart_guide_v1-21.pdf

{#library
  library('lubridate')
  library('sqldf')
  library('reshape')
  library('stringr')
  library('RJSONIO')
}# libraries


{ # downloading and importing data
  # load(".RData")
  # 
#   link_question_summary = 'http://datamart.scicast.org/question/?format=csv&api_key=API_KEY_FROM_SCICAST'
#   download.file(url=link_question_summary, destfile='question_summary.csv', method='wget')
  question_summary <- read.csv("question_summary.csv", na.strings="None", stringsAsFactors=FALSE)
#   
#   
#   link_question_history = 'http://datamart.scicast.org/question_history/?format=csv&api_key=API_KEY_FROM_SCICAST'
#   download.file(url=link_question_history, destfile='question_history.csv', method='wget')
  question_history <- read.csv("question_history.csv", na.strings="None", stringsAsFactors=FALSE)
#   
#   
#   link_person = 'http://datamart.scicast.org/person/?format=csv&api_key=API_KEY_FROM_SCICAST'
#   download.file(url=link_person, destfile='person.csv', method='wget')
  person <- read.csv("person.csv", na.strings="None", stringsAsFactors=FALSE)
#   
#   link_trade_history = 'http://datamart.scicast.org/trade_history/?format=csv&api_key=API_KEY_FROM_SCICAST'
#   download.file(url=link_trade_history, destfile='trade_history.csv', method='wget')
  trade_history <- read.csv("trade_history.csv", na.strings="None", stringsAsFactors=FALSE)
#   
#   
#   link_comment = 'http://datamart.scicast.org/comment/?format=csv&api_key=API_KEY_FROM_SCICAST'
#   download.file(url=link_comment, destfile='comment.csv', method='wget')
  comment <- read.csv("comment.csv", na.strings="None", stringsAsFactors=FALSE)
#   
#   link_leaderboard = 'http://scicast.org:8200/person/leaderboard/?format=csv&api_key=API_KEY_FROM_SCICAST&start_date=05-26-2015'
#   download.file(url=link_leaderboard, destfile='leaderboard.csv', method='wget')
  leaderboard <- read.csv("leaderboard.csv", na.strings="None", stringsAsFactors=FALSE)
  # 
  # rm(link_comment, link_leaderboard, link_person, link_question_history, link_question_summary, link_trade_history)
}# downloading and importing data

{# date conversion, getting max dates
  # 
  date_question_history = trade_history
  # date_question_history$traded_at = as.Date(substr(trade_history$traded_at, 0, 10), "%Y-%m-%d")
  
  # date_question_history$traded_at = as.POSIXlt(trade_history$traded_at)
  date_question_history$traded_at = as.numeric(as.POSIXct(trade_history$traded_at))
  
  #getting last trade information:
  last_trade = sqldf("select                     
                     question_id,
                     trade_id,
                     choice_index,
                     interface_type,
                     max(traded_at) traded_at,
                     new_value_list
                     from date_question_history
                     group by
                     question_id                    
                     ")
}# date conversion, getting max dates

{ # merging the datasets
  merged_questions = merge(last_trade, question_summary, by.x = "question_id", by.y = "question_id")
  
  merged_questions = sqldf("select                     
                          *
                          from merged_questions
                          where resolution_index is NOT NULL
                          AND (classification LIKE '%binary%'
                          OR classification LIKE '%unordered%')
                          ")   #this is to select only multinomial questions                    
  
  
  sort_leaderboard = leaderboard[order(-leaderboard$max_score),]
  sort_leaderboard = sort_leaderboard[1:39,]
}# merging the datasets

{# getting spark data from json file
  download.file(url='http://datamart.scicast.org/question/?format=json&api_key=API_KEY_FROM_SCICAST',
                destfile='spark.json'
  )
  spark = fromJSON('spark.json')
  
  spark_data = data.frame()
  spark_temp = c()
  
  for (i in 1:length(spark)){
    spark_temp = c()
    spark_question = data.frame( question_id = numeric())
    #   print(i)
    for (j in 1:length(spark[i][[1]]$spark_collaborators)){  
      spark_temp = c(spark_temp, spark[i][[1]]$spark_collaborators[[j]]$predict_id)
    }
    spark_question[1,] = spark[i][[1]]$question_id #howto
    spark_question$id_lists[[1]] = if(is.null(spark_temp)){NA} else{spark_temp}
    spark_data = rbind(spark_data, spark_question)
  }
  rm(i, j, spark_question, spark_temp, spark)
}# getting spark data from json file

{#looping over each question
  numtrades = 400# 50
  final = data.frame()
  t = now()
  
  num_questions = nrow(merged_questions)
  
  for(i in 1:num_questions) {
    id = merged_questions$question_id[i]
    cat(paste(num_questions-i, " "))  
{# preparing the data: sql_history, prize_question, half-time, etc
      all_choice_values = data.frame()
      question_trade_volume = 0
      
      
      question_sub = subset(merged_questions, merged_questions$question_id == id)
      
      sql_trade_history =  fn$sqldf(" select                     
                                    question_id,
                                    choice_index,
                                    traded_at,
                                    interface_type,
                                    new_value_list,
                                    old_value_list,
                                    user_id,
                                    assets_per_option
                                    from date_question_history
                                    where question_id = $id
                                    order by traded_at
                                    limit $numtrades
                                    ")
      
      sql_uncertainty =  fn$sqldf(" select                     
                                  question_id,
                                  count(*) count,                             
                                  max(traded_at) last_trade
                                  from date_question_history
                                  where question_id = $id
                                  group by question_id
                                  ")
      
      lastrowcount = floor(0.9*sql_uncertainty$count)
      #print(paste("last row=",lastrowcount))
      sql_last_row = fn$sqldf("select                     
                              question_id,
                              new_value_list,
                              traded_at
                              from date_question_history
                              where question_id = $id
                              order by traded_at
                              limit $lastrowcount
                              ")
      
      question_num_trades = sql_uncertainty$count
      
      halftime = as.numeric(as.POSIXct(question_sub$created_at)) + 0.5*(sql_uncertainty$last_trade -  as.numeric(as.POSIXct(question_sub$created_at)) )
      
      sql_half_time = fn$sqldf("select                     
                               question_id,                
                               new_value_list,
                               abs(traded_at - $halftime)/3600/24 AS distance_from_test
                               from date_question_history
                               where question_id = $id
                               order by distance_from_test
                               limit 1
                               ")
      
      
      prize_question = if( length(grep("prize", tolower(subset(question_summary, question_summary$question_id == id)$categories))) > 0){1} else{0}
      #   print(prize_question)
      is_visible = 5 - nchar(subset(question_summary, question_summary$question_id == id)$is_visible)
    }# preparing the data: sql_history, prize_question, half-time, etc


for (c in 1:length(as.numeric(strsplit(merged_questions$new_value_list[i], ',')[[1]])) ) { 
{
  each_choice_values = data.frame(
    choice_happened = numeric(),
    prob_choice_final = numeric(),
    prize_question = numeric(),
    is_visible= numeric(),
    num_choices= numeric(),
    crowd_predicted_final= numeric(),
    crowd_predicted_90pct= numeric(),
    choice = numeric(),
    resolution_value_array_choice = numeric(),
    prob_after_90pct_of_trades = numeric(),
    prob_at_half_market = numeric(),
    question_num_trades = numeric(),
    question_trade_volume = numeric(),
    asset_trade_volume = numeric()
  )
}# creating the empty choice values

{#creating choice column names 
  trades_long_orig = data.frame(
    "prob_choice" = NA,
    "prob_choice_change" = NA,
    "assets_choice" = NA,
    "power_trades_choice" = NA,
    "leaderboard_trades_choice" = NA,
    "spark_trade_choice" = NA,
    "prob_choice_average" = NA
  )
  
  trades_long_final = data.frame(
    "prob_choice1" = NA,
    "prob_choice_change1" = NA,
    "assets_choice1" = NA,
    "power_trades_choice1" = NA,
    "leaderboard_trades_choice1" = NA,
    "spark_trade_choice1" = NA,
    "prob_choice_average1" = NA
  )
  trades_long_temp = trades_long_orig
  
  asset_trade_volume = 0 
  
  for (n in c(2:4, 1:floor(numtrades/5)*5) ){
    
    names(trades_long_temp) <- paste(names(trades_long_orig), n, sep = "")
    trades_long_final = cbind(trades_long_final,trades_long_temp)           
  }
  rm(trades_long_orig, trades_long_temp, n)
}#creating choice column names 

{# vertical trade history to horizontal column names in final data
  average_choice = c()
  for (k in 1:numtrades){
    #     for (k in c(1:4, 1:floor(numtrades/5)*5){
    if (k <= dim(sql_trade_history)[1]){
      average_choice = c(average_choice, as.numeric(strsplit(sql_trade_history$new_value_list[k], ',')[[1]])[c])
      if (k %in% c(1:4, 1:floor(numtrades/5)*5)){
        trades_long_final[[paste("prob_choice", k, sep="")]] = as.numeric(strsplit(sql_trade_history$new_value_list[k], ',')[[1]])[c]
        trades_long_final[[paste("prob_choice_change", k, sep="")]] = as.numeric(strsplit(sql_trade_history$new_value_list[k], ',')[[1]])[c] - as.numeric(strsplit(sql_trade_history$old_value_list[k], ',')[[1]])[c]
        trades_long_final[[paste("power_trades_choice", k, sep="")]] = sql_trade_history$interface_type[k]
        trades_long_final[[paste("leaderboard_trades_choice", k, sep="")]] =  if (sql_trade_history$user_id[k] %in% sort_leaderboard$user_id) {1} else {0}
        trades_long_final[[paste("assets_choice", k, sep="")]] = as.numeric(strsplit(substr(sql_trade_history$assets_per_option[k], 2, nchar(sql_trade_history$assets_per_option[k])-1), ',')[[1]])[c]
        trades_long_final[[paste("spark_trade_choice", k, sep="")]] = if (sql_trade_history$user_id[k] %in% subset(spark_data, spark_data$question_id == id)$id_lists) {1} else {0}                
        trades_long_final[[paste("prob_choice_average", k, sep="")]] = mean(average_choice)
        
        average_choice = c()
      }
    }
  }
  rm(average_choice)
}

prob_choice_final = as.numeric(strsplit(question_sub$new_value_list, ',')[[1]])[c]
choice_happened = as.numeric(question_sub$resolution_index == (c-1))
prob_after_90pct_of_trades = if(lastrowcount>0){as.numeric(strsplit(sql_last_row$new_value_list[lastrowcount], ',')[[1]])[c]} else{NA}
prob_at_half_market = as.numeric(strsplit(sql_half_time$new_value_list, ',')[[1]])[c]
resolution_value_array_choice =  as.numeric(strsplit(substr(question_sub$resolution_value_array, 2, nchar(question_sub$resolution_value_array)-1), ',')[[1]])[question_sub$resolution_index+1] #this is just to say if this was a scaled winning term, question_level value

for(i in 1:nrow(sql_trade_history)) {
  asset_trade_volume = asset_trade_volume + 
    abs(
      as.numeric(strsplit(substr(sql_trade_history$assets_per_option[i], 2, nchar(sql_trade_history$assets_per_option[i])-1), ',')[[1]])[c]
    )
  #       }   
}# all assets traded for this choice


each_choice_values[1,] = c(
  choice_happened,
  prob_choice_final,
  prize_question,
  is_visible,
  num_choices = length(as.numeric(strsplit(question_sub$new_value_list, ',')[[1]])), #length(as.numeric(strsplit(merged_questions$new_value_list[i], ',')[[1]])),
  crowd_predicted_final = which.max(as.numeric(strsplit(question_sub$new_value_list, ',')[[1]])) == c,
  crowd_predicted_90pct = if(lastrowcount>0){ which.max(as.numeric(strsplit(sql_last_row$new_value_list[lastrowcount], ',')[[1]])) == c } else{NA},
  c-1, #just a simple choice indicator in the table
  resolution_value_array_choice,
  prob_after_90pct_of_trades,
  prob_at_half_market,
  question_num_trades,
  question_trade_volume,
  asset_trade_volume     
)


all_choice_values = rbind(all_choice_values,cbind(question_sub, each_choice_values, trades_long_final))

rm(trades_long_final,c, num_assets, asset_trade_volume, num_assets_question, num_trades_total_50, num_power_trades_total_50, each_choice_values,num_leaderboard_trades_total_50, num_trades_total_50)
} #choice loop ends

all_choice_values$question_trade_volume = sum(all_choice_values$asset_trade_volume)


final = rbind(final, all_choice_values)


rm(sql_half_time, c, lastrowcount,all_choice_values, time, x, sql_last_row, halftime, is_visible, prize_question, question_num_trades, question_trade_volume)


  } # question loop ends


print(Sys.time() - t)
rm(question_sub, num_questions,sql_uncertainty, question_trade_volume, prob_after_90pct_of_trades, prob_at_half_market, id,j, t, lifespan_question, timebreak, prob, i, fraction, numtrades, choice_happened, k, leaderboard, name_choice, num_trades, prob_after_90percent_of_trades, prob_choice_final, resolution_value_array_choice) 

row.names(final) = NULL

}# #looping over each question

save.image(".RData")
write.csv(final, 'prediction_market_computed.csv')

In [33]:
# me@nikete.com, dhaval@mit.edu
# ML code for testing price information integration

%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn_pandas import DataFrameMapper, cross_val_score
import sklearn.preprocessing, sklearn.decomposition, sklearn.ensemble, sklearn.cross_validation, sklearn.linear_model, sklearn.pipeline, sklearn.metrics, sklearn.gaussian_process   
from patsy import dmatrices 

df = pd.read_csv("prediction_market_computed.csv")

def feature_list(n, only_last_trade = False):
    s = "choice_happened ~ "
    if only_last_trade == True:
        numtrades_list = [n]
    else:
        numtrades_list = [1,2,3,4] + map(lambda x: x*5, range(1,n/5))
    for d in numtrades_list:
        s +="+ prob_choice%d   "%(d)
#         s +="+ prob_choice%d + prob_choice_average%d "%(d) # use this if you want to include prob_choice_average. modify to include other features 
    return s


def model_crowd(data, model, specification, n_folds = 5):
    y, X = dmatrices(specification, data, NA_action='drop')
    y=y.ravel()
    skf = sklearn.cross_validation.StratifiedKFold(y, n_folds)
    scores=[]
    for train_index, test_index in skf:
        X_train, y_train = X[train_index], y[train_index]
        X_test, y_test   = X[test_index],  y[test_index]        
        model = model.fit(X_train, y_train)
        scores.append( model.score(X_test, y_test))        
    return np.mean(scores)

print 'mean prediction score for only last trade:',  model_crowd(df, sklearn.linear_model.LogisticRegression(), feature_list(150, True))
print 'mean prediction score all but last trade :', model_crowd(df, sklearn.linear_model.LogisticRegression(), feature_list(150, False))

mean prediction score for only last trade: 0.911669872768
mean prediction score all but last trade : 0.903109243697
